# 1. Selenium이란?
* 웹브라우저를 자동으로 제어하는 라이브러리
* 원래 다양한 웹브라우저를 자동으로 테스트하는 테스트 도구
* 동적 웹사이트에서 정보를 가져오는데 활용

In [234]:
# !pip install selenium
# !pip install webdriver-manager

In [312]:
import selenium
print(selenium.__version__)

4.31.1


In [313]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import time

# 크롬 옵션즈에 정보를 담아 사람인 것 처럼 만들기
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ['enable-logging'])
options.add_argument("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36")
options.add_argument('lang=ko_KR')

# 크롬 웹브라우저 드라이버 자동 다운로드
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.set_window_size(1920, 1080) #웹브라우저 해상도 조절
driver.get("https://search.shopping.naver.com/book/home")

In [314]:
#lnb > div > div._lnbSearch_lnb_search_10s9T > div > div > form > input
# _lnbSearch_search_input_YVyzR N=a:sech.input
serch_box = driver.find_element(By.CSS_SELECTOR, "#lnb > div > div._lnbSearch_lnb_search_10s9T > div > div > form > input")
serch_box.send_keys("핀테크") #+Keys.ENTER)
# serch_box.send_keys("핀테크"+Keys.ENTER) 핀테크 검색됨

In [315]:
#lnb > div > div._lnbSearch_lnb_search_10s9T > div > div > form > button
search_button = driver.find_element(By.CSS_SELECTOR, "#lnb > div > div._lnbSearch_lnb_search_10s9T > div > div > form > button") #돋보기 버튼
search_button.click() #검색이 되는 것

In [316]:
page_html = driver.page_source
soup = bs(page_html, 'lxml')
book_list = soup.select(".list_book > li")
for book in book_list:
    title = book.select_one("span.bookListItem_text__SL9m9").text # 책 제목
    author = book.select("span.bookListItem_define_data__IUMgt")[0].text # 저자
    publisher = book.select("span.bookListItem_define_data__IUMgt")[1].text # 출판사
    pub_date = book.select_one("div.bookListItem_detail_date__s7KQe").text.strip(". ").replace(".","-") # 출간일
    if book.select("span.bookPrice_price__OagxI") != None:
        print("가격부분:",len(book.select("span.bookPrice_price__OagxI")))
        if len(book.select("span.bookPrice_price__OagxI")) == 2:
            price = book.select("span.bookPrice_price__OagxI")[0].text.split(" ")[1].replace(",","").replace("원","") # 종이책 가격
            eb_price = book.select("span.bookPrice_price__OagxI")[1].text.split(" ")[1].replace(",","").replace("원","") # e북 가격
        elif len(book.select("span.bookPrice_price__OagxI")) == 1:
            price = book.select("span.bookPrice_price__OagxI")[0].text.split(" ")[1].replace(",","").replace("원","") # 종이책 가격
            eb_price = 0
    else:
        price = 0
        eb_price = 0
    
    print(title, author, publisher, pub_date, price, eb_price)

가격부분: 2
핀테크 한석주 커뮤니케이션북스 2015-11-01 11400 8640
가격부분: 1
핀테크 (IT와 금융이 만나는 새로운 세상) 강창호 한빛미디어 2015-06-05 12600 0
가격부분: 2
핀테크 전쟁 (세계 금융 시장의 거대한 트렌드 | 새로운 돈의 시대가 온다) 브렛 킹 예문 2015-06-12 13500 9450
가격부분: 1
핀테크 4.0 (핀테크 혁명과 금융의 미래) 김종현 한국금융연수원 2022-05-24 17100 0
가격부분: 1
핀테크 혁명 (모바일 은행 설립자가 알려주는 핀테크 시대의 돈 관리 기술) 앤 보덴 유엑스리뷰 2020-03-03 20700 0
가격부분: 2
핀테크와 법 (제3판) 강현구, 유주선 외 1명 씨아이알 2020-11-12 19800 13860
가격부분: 1
핀테크가 강한 나라 (석학 9인의 금융 플랫폼 미래 통찰) 강형구, 김솔 외 3명 한울아카데미 2024-04-08 35100 0
가격부분: 1
내일의 핀테크 (금융대혁명의 시대, 어떻게 살아남을 것인가?) 홍장원 매경출판 2016-02-22 11700 0
가격부분: 0
핀테크 시대 (기술변화와 함께 다가오는 법과 규제 패러다임의 변화) 고학수, 권남훈 외 3명 박영사 2015-12-05 11700 0
가격부분: 1
차이나 핀테크 (중국에선 현금이 필요없다) 구자근 스리체어스 2018-02-20 10800 0


# 마우스를 스크롤해서 40개 정보 가져오기
* window.scrollTo({시작위치}, {step})

In [265]:
import pandas as pd

In [318]:
final_result = pd.DataFrame()

for page in range(1,7):
    #마우스 스크롤로 40개 정보 가져옴
    y = 0
    num = 0
    y_step = 1000
    for scroll_times in range(1,7):
        y += y_step
        driver.execute_script(f"window.scrollTo({0}, {y})") #윈도우를 웹브라우저 절대위치 1부터 1000까지 스크롤해라
        time.sleep(1) #2초마다 스크롤 아래로
        #이제 페이지 넘겨서 copy 하는 코드 필요
    page_html = driver.page_source 
    soup = bs(page_html, 'lxml') #html 페이지 parsing
    book_list = soup.select(".list_book > li") #정보 추출

    result = {}
    
    for book in book_list:
        num +=1
        title = book.select_one("span.bookListItem_text__SL9m9").text # 책 제목
        #저자
        if len(book.select("span.bookListItem_define_data__IUMgt")) == 0:
            author = "None"   
            publisher = "None"   
        elif len(book.select("span.bookListItem_define_data__IUMgt")) == 1:
            if len(book.select("span.bookListItem_define_data__IUMgt")[0]) ==0 :
                author = "None"  
                publisher = book.select("span.bookListItem_define_data__IUMgt")[1].text 
            else:
                author = book.select("span.bookListItem_define_data__IUMgt")[0].text 
                publisher = "None"    
        else:
            author = book.select("span.bookListItem_define_data__IUMgt")[0].text 
            publisher = book.select("span.bookListItem_define_data__IUMgt")[1].text
        #출간일
        if book.select_one("div.bookListItem_detail_date__s7KQe")  != None:
            pub_date = book.select_one("div.bookListItem_detail_date__s7KQe").text.strip(". ").replace(".","-") 
            
        else:
             pub_date = "None"
        #가격
        if book.select("span.bookPrice_price__OagxI") != None:
            # print("가격부분:",len(book.select("span.bookPrice_price__OagxI")))
            if len(book.select("span.bookPrice_price__OagxI")) == 2:
                price = book.select("span.bookPrice_price__OagxI")[0].text.split(" ")[1].replace(",","").replace("원","") # 종이책 가격
                eb_price = book.select("span.bookPrice_price__OagxI")[1].text.split(" ")[1].replace(",","").replace("원","") # e북 가격
            # elif book.select_one("span.n_npay_info__xZATR") == "네이버페이":
            #     pass
            elif len(book.select("span.bookPrice_price__OagxI")) == 1:
                price = book.select_one(f"#book_list > ul > li:nth-child({num}) > div > div > div > span > em").text #셀때마다 num 카운트되어 nthchild로 받음
                eb_price = 0
        else:
            price = 0
            eb_price = 0
        keys = ( "title","author","publisher","pub_date","price","eb_price" )
        data = ( title,author,publisher,pub_date,price,eb_price )
        for k,i in zip(keys,data):
            result.setdefault(k, []).append(data)
    result_df = pd.DataFrame(result)        
    final_result = pd.concat([final_result, result_df])
    next_page_button=  driver.find_element(By.CSS_SELECTOR, "div.Paginator_list_paging__hUKRc > button.Paginator_btn_next__7NiBL")
    next_page_button.click()

In [319]:
final_result

,title,author,publisher,pub_date,price,eb_price
0,"(핀테크, 한석주, 커뮤니케이션북스, 2015-11-01, 11400, 8640)","(핀테크, 한석주, 커뮤니케이션북스, 2015-11-01, 11400, 8640)","(핀테크, 한석주, 커뮤니케이션북스, 2015-11-01, 11400, 8640)","(핀테크, 한석주, 커뮤니케이션북스, 2015-11-01, 11400, 8640)","(핀테크, 한석주, 커뮤니케이션북스, 2015-11-01, 11400, 8640)","(핀테크, 한석주, 커뮤니케이션북스, 2015-11-01, 11400, 8640)"
1,"(핀테크 (IT와 금융이 만나는 새로운 세상), 강창호, 한빛미디어, 2015-06...","(핀테크 (IT와 금융이 만나는 새로운 세상), 강창호, 한빛미디어, 2015-06...","(핀테크 (IT와 금융이 만나는 새로운 세상), 강창호, 한빛미디어, 2015-06...","(핀테크 (IT와 금융이 만나는 새로운 세상), 강창호, 한빛미디어, 2015-06...","(핀테크 (IT와 금융이 만나는 새로운 세상), 강창호, 한빛미디어, 2015-06...","(핀테크 (IT와 금융이 만나는 새로운 세상), 강창호, 한빛미디어, 2015-06..."
2,"(핀테크 전쟁 (세계 금융 시장의 거대한 트렌드 | 새로운 돈의 시대가 온다), 브...","(핀테크 전쟁 (세계 금융 시장의 거대한 트렌드 | 새로운 돈의 시대가 온다), 브...","(핀테크 전쟁 (세계 금융 시장의 거대한 트렌드 | 새로운 돈의 시대가 온다), 브...","(핀테크 전쟁 (세계 금융 시장의 거대한 트렌드 | 새로운 돈의 시대가 온다), 브...","(핀테크 전쟁 (세계 금융 시장의 거대한 트렌드 | 새로운 돈의 시대가 온다), 브...","(핀테크 전쟁 (세계 금융 시장의 거대한 트렌드 | 새로운 돈의 시대가 온다), 브..."
3,"(핀테크 4.0 (핀테크 혁명과 금융의 미래), 김종현, 한국금융연수원, 2022-...","(핀테크 4.0 (핀테크 혁명과 금융의 미래), 김종현, 한국금융연수원, 2022-...","(핀테크 4.0 (핀테크 혁명과 금융의 미래), 김종현, 한국금융연수원, 2022-...","(핀테크 4.0 (핀테크 혁명과 금융의 미래), 김종현, 한국금융연수원, 2022-...","(핀테크 4.0 (핀테크 혁명과 금융의 미래), 김종현, 한국금융연수원, 2022-...","(핀테크 4.0 (핀테크 혁명과 금융의 미래), 김종현, 한국금융연수원, 2022-..."
4,"(핀테크 혁명 (모바일 은행 설립자가 알려주는 핀테크 시대의 돈 관리 기술), 앤 ...","(핀테크 혁명 (모바일 은행 설립자가 알려주는 핀테크 시대의 돈 관리 기술), 앤 ...","(핀테크 혁명 (모바일 은행 설립자가 알려주는 핀테크 시대의 돈 관리 기술), 앤 ...","(핀테크 혁명 (모바일 은행 설립자가 알려주는 핀테크 시대의 돈 관리 기술), 앤 ...","(핀테크 혁명 (모바일 은행 설립자가 알려주는 핀테크 시대의 돈 관리 기술), 앤 ...","(핀테크 혁명 (모바일 은행 설립자가 알려주는 핀테크 시대의 돈 관리 기술), 앤 ..."
...,...,...,...,...,...,...
35,(교보문고 교보문고 미래유망산업분야 기술용어사전 - 인공지능 바이오 핀테크 빅데이터...,(교보문고 교보문고 미래유망산업분야 기술용어사전 - 인공지능 바이오 핀테크 빅데이터...,(교보문고 교보문고 미래유망산업분야 기술용어사전 - 인공지능 바이오 핀테크 빅데이터...,(교보문고 교보문고 미래유망산업분야 기술용어사전 - 인공지능 바이오 핀테크 빅데이터...,(교보문고 교보문고 미래유망산업분야 기술용어사전 - 인공지능 바이오 핀테크 빅데이터...,(교보문고 교보문고 미래유망산업분야 기술용어사전 - 인공지능 바이오 핀테크 빅데이터...
36,([보리보리/아이와함께]NEW Live 과학 첨단과학 15 - 핀테크와 가상화폐 (...,([보리보리/아이와함께]NEW Live 과학 첨단과학 15 - 핀테크와 가상화폐 (...,([보리보리/아이와함께]NEW Live 과학 첨단과학 15 - 핀테크와 가상화폐 (...,([보리보리/아이와함께]NEW Live 과학 첨단과학 15 - 핀테크와 가상화폐 (...,([보리보리/아이와함께]NEW Live 과학 첨단과학 15 - 핀테크와 가상화폐 (...,([보리보리/아이와함께]NEW Live 과학 첨단과학 15 - 핀테크와 가상화폐 (...
37,(교보문고 교보문고 핀테크 산업분야 취업가이드 - 네이버 카카오뱅크 카카오페이 KG...,(교보문고 교보문고 핀테크 산업분야 취업가이드 - 네이버 카카오뱅크 카카오페이 KG...,(교보문고 교보문고 핀테크 산업분야 취업가이드 - 네이버 카카오뱅크 카카오페이 KG...,(교보문고 교보문고 핀테크 산업분야 취업가이드 - 네이버 카카오뱅크 카카오페이 KG...,(교보문고 교보문고 핀테크 산업분야 취업가이드 - 네이버 카카오뱅크 카카오페이 KG...,(교보문고 교보문고 핀테크 산업분야 취업가이드 - 네이버 카카오뱅크 카카오페이 KG...
38,"(핀테크에반젤리스트 - 디지털 금융, 핀테크 스타트업, 금융 혁신, 핀테크 생태계,...","(핀테크에반젤리스트 - 디지털 금융, 핀테크 스타트업, 금융 혁신, 핀테크 생태계,...","(핀테크에반젤리스트 - 디지털 금융, 핀테크 스타트업, 금융 혁신, 핀테크 생태계,...","(핀테크에반젤리스트 - 디지털 금융, 핀테크 스타트업, 금융 혁신, 핀테크 생태계,...","(핀테크에반젤리스트 - 디지털 금융, 핀테크 스타트업, 금융 혁신, 핀테크 생태계,...","(핀테크에반젤리스트 - 디지털 금융, 핀테크 스타트업, 금융 혁신, 핀테크 생태계,..."


In [ ]:
#container > div > div.bookSearch_book_search__Fm5Em > div > div.Paginator_list_paging__hUKRc > button
#book_list > ul > li:nth-child(40) > div > div > div > div > span
#container > div > div.bookSearch_book_search__Fm5Em > div > div.Paginator_list_paging__hUKRc > button
#container > div > div.bookSearch_book_search__Fm5Em > div > div.Paginator_list_paging__hUKRc > button > svg

In [174]:
npay = soup.select_one("#book_list > ul > li:nth-child(40) > div > div > div > div > span > span > span > span").text
npay
# if "네이버페이" in L:
#     print("있음")
# else:
#     print("없음")

'네이버페이'

In [54]:
#book_list > ul > li:nth-child(40) > div > div > div > span
#book_list > ul > li:nth-child(40) > div > div > div > span > em
#book_list > ul > li:nth-child(40) > div > div > div > span > em
#book_list > ul > li:nth-child(40) > div > div > div > div > span
#book_list > ul > li:nth-child(40) > div > div > div > div > span > span
#book_list > ul > li:nth-child(40) > div > div > div > div > span > span > span > span
#book_list > ul > li:nth-child(40) > div > div > div > span
#book_list > ul > li:nth-child(40) > div > div > div > span > em
#book_list > ul > li:nth-child(40) > div > div > div > span > em
#book_list > ul > li:nth-child(40) > div > div > div > span > span
#book_list > ul > li:nth-child(39) > div > div > div > span > em

In [158]:
price = book.select("span.bookPrice_price__OagxI").text.replace("원","")
print(price)

AttributeError: ResultSet object has no attribute 'text'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [212]:
price = soup.select_one("#book_list > ul > li:nth-child(40) > div > div > div > span > em")
print(price)
# print(len(price))

<em>2,790</em>


In [191]:
np = soup.select("span.bookPrice_price__OagxI")[40]
print(len(np)

3


In [196]:
np2 = soup.select("span.bookPrice_price__OagxI")[1]
print(len(np2))

3


In [230]:
auth = soup.select("span.bookListItem_define_data__IUMgt")[0].text
auth

'씨익북스 편집부'